In [ ]:
%matplotlib notebook

# Notebook simulations - DOF / perceptron une couche

Ce notebook présente les simulations de calcul de degrés de libertés pour des données non linéaires générées par un polynôme.

Deux modèles sont présentés :
1. Un modèle linéaire en paramètres dont la dimension d'entrée est augmentée,
2. Un réseau de neurone (perceptron) à une couche cachée.

Le notebook requiert la création d'un dossier "Dossier_resultats" pour fonctionner.
Ce dernier contiendra les résultats obtenus en format csv (contient des NaN correspondants aux colonnes moins longues que d'autres, à retirer lors d'analyses).



### Génération de données

Imports

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_squared_error as MSE

Génération du polynôme pour le premier cas linéaire puis pour le perceptron.

In [ ]:
# Set seed
seed = 0
np.random.seed(0)

# Number of samples
n      = 10000          # independent samples
n_mc   = 100           # Number of data-sets for output
# Number of features
d      = 2
# Noise variance
sigma_2= 0.1
epsilon= np.sqrt(sigma_2)*np.random.randn(n,n_mc)
# Regularization for both modelizations
l1_reg = 0.1
l2_reg = [1e-12, 1e-9, 1e-6, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 1e0, 1e1, 2e1, 5e1, 1e2, 1e3]

In [ ]:
np.random.seed(0)

# Feature matrix
X        = 2*np.random.random((n,d))-1
poly     = PolynomialFeatures(5)
X_transf = poly.fit_transform(X)

# Matrix containing the observations with polynomial features
Y        = np.tile(0.1*np.power(X[:, 0], 5) \
                   - 0.4*np.power(X[:,0], 2)*np.power(X[:,1], 2) \
                   + 0.8*np.power(X[:,0], 2)*X[:,1] \
                   - 0.01*np.power(X[:,1], 3) \
                   +0.4*X[:,1],                                    # Max order (5), then 4 / 3 / 3 / 1
                   (n_mc, 1)).T                                    # Copy n_mc times - size n x n_mc
Y        = Y + epsilon

### Prints
print("Number of features  : {}  / number of monomials : {}.".format(X.shape[1], X_transf.shape[1]))
print("Input shape (lin)   :", X.shape)
print("Input shape (nn-lin):", X_transf.shape)
print("Output shape        :", Y[:,0].shape)

In [ ]:
nplot = 100

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(X[:nplot,0], X[:nplot,1], Y[:nplot,0], marker="o")
ax.set_xlabel('x_1');
ax.set_ylabel('x_2');
ax.set_zlabel('y');

### Modèle linéaire

Pour une régularisation L2, solution du problème d'optimisation :
$$
L(\boldsymbol{\theta}) = \lVert \mathbf{X}\boldsymbol{\theta} - \mathbf{y} \rVert^2_2 + \eta \lVert \boldsymbol{\theta} \rVert^2_2.
$$
La solution est :
$$
\hat{\boldsymbol{\theta}} = (\mathbf{X}^\top\mathbf{X} + \eta\mathbf{I})^{-1}\mathbf{X}^\top \mathbf{y}.
$$

In [ ]:
# Select data-set between 1 and n_mc
ds = 0
# Save dofs & MSE
dfs = []
mses = []
# Optimize model with L2 penalty
for eta in l2_reg:
    inv_ = np.linalg.inv(np.dot(X_transf.T, X_transf) + eta*np.eye(X_transf.shape[1]))
    hat_theta = np.dot(inv_, np.dot(X_transf.T, Y[:, ds:(ds+1)]))
                
    # Compute outputs
    hat_y = np.dot(X_transf, hat_theta)
    
    # Compute MSE
    mses.append(MSE(Y[:,ds], hat_y))
    # Compute dofs
    H = np.dot(X_transf.T, X_transf)     # Hessian without penalty
    vp, _ = np.linalg.eig(H)             # get eignevalues
    df_lin = np.sum(np.divide(vp, vp + eta))
    dfs.append(df_lin)

Calcul des degrés de liberté

In [ ]:
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(l2_reg, dfs, label="Degrees of freedom")
plt.plot(l2_reg, hat_theta.shape[0]*np.ones(len(l2_reg)), 'r', label="Nb params")
plt.legend()
plt.xscale('log')

plt.subplot(2, 1, 2)
plt.plot(l2_reg, mses, label="MSE")
plt.plot(l2_reg, sigma_2*np.ones(len(l2_reg)), 'r', label="Noise variance")
plt.legend()
plt.xscale('log')

# Save figures to csv
dict_graph = {}

dict_graph["l2_eta"]    = np.array(l2_reg)
dict_graph["dofs"]      = np.array(dfs)
dict_graph["MSE"]       = np.array(mses)
dict_graph["var"]       = np.array(sigma_2*np.ones(len(l2_reg)))
dict_graph["nb_params"] = np.array(hat_theta.shape[0]*np.ones(len(l2_reg)))

df = pd.DataFrame.from_dict(dict_graph, orient = "index")
df = df.T
df.to_csv("Dossier_resultats/lin_poly_degree_5_L2.csv",
          na_rep="nan", index=False)

#### Calcul du critère AIC pour différents ordres de polynômes sans régularisation

In [ ]:
degrees = [1, 2, 3, 4, 5, 6, 7, 8]

ds  = 0 # Select data-set between 1 and n_mc
eta = 0 # no regularization
# Save data
aic_poly = [];  bic_poly = [];
mse_poly = []; feat_poly = [];

# Compute for order 1
inv_ = np.linalg.inv(np.dot(X.T, X) + eta*np.eye(X.shape[1]))
hat_theta = np.dot(inv_, np.dot(X.T, Y[:, ds:(ds+1)]))
hat_y = np.dot(X, hat_theta)
aic_poly.append(n*np.log(MSE(Y[:,0], hat_y)) + 2*X.shape[1])
bic_poly.append(n*np.log(MSE(Y[:,0], hat_y)) + np.log(n)*X.shape[1])
mse_poly.append(MSE(Y[:,0], hat_y))
feat_poly.append(X.shape[1])
print("Degree {} : {} features".format(1, X.shape[1]))

# Compute for higher order
for deg in degrees[1:]:
    poly_tmp = PolynomialFeatures(deg)
    X_tmp    = poly_tmp.fit_transform(X)
    # Solve opt.
    inv_ = np.linalg.inv(np.dot(X_tmp.T, X_tmp) + eta*np.eye(X_tmp.shape[1]))
    hat_theta = np.dot(inv_, np.dot(X_tmp.T, Y[:, ds:(ds+1)]))
    hat_y = np.dot(X_tmp, hat_theta)
    aic_poly.append(n*np.log(MSE(Y[:,0], hat_y)) + 2*X_tmp.shape[1])
    bic_poly.append(n*np.log(MSE(Y[:,0], hat_y)) + np.log(n)*X_tmp.shape[1])
    mse_poly.append(MSE(Y[:,0], hat_y))
    
    feat_poly.append(X_tmp.shape[1])
    print("Degree {} : {} features".format(deg, X_tmp.shape[1]))   

In [ ]:
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(degrees, aic_poly, label="AIC")
plt.plot(degrees, bic_poly, 'r', label="BIC")
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(degrees, mse_poly, label="MSE")
plt.xlabel("Degrees")
plt.legend()

# Save figures to csv
dict_graph = {}

dict_graph["degrees"]  = np.array(degrees)
dict_graph["AIC"]      = np.array(aic_poly)
dict_graph["BIC"]      = np.array(bic_poly)
dict_graph["MSE"]      = np.array(mse_poly)
dict_graph["features"] = np.array(feat_poly)

df = pd.DataFrame.from_dict(dict_graph, orient = "index")
df = df.T
df.to_csv("Dossier_resultats/lin_poly_degrees_AIC.csv",
          na_rep="nan", index=False)



### Perceptron

Imports - Utilisation de Keras.

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD

from tensorflow.random import set_seed

from scipy.linalg import cho_factor, cho_solve

Définition des fonctions pour le calcul des dofs (hessienne et gradient) pour une somme des erreurs quadratiques (SEQ).

In [ ]:
def sig(x):    # Activation function
    return np.tanh(x)
def dsig(x):   # 1st Derivative of act. func.
    return np.ones_like(x) - np.power(sig(x), 2)
def ddsig(x):  # 2nd Derivative of act. func.
    return -2*sig(x)*dsig(x)

def hessian_i(theta, X, y):
    # Check if one sample is given
    if X.shape[0] > 1:
        print("Give one sample to the function with")
        print(" shape (1, p),")
        print(" where p is the number of features.")
        X = X[0:1, :]
        y = y[0].reshape(-1, 1)
    # Compute gradient
    grady = grady_i(theta, X, y)
    # Get parameters
    W1 = theta[0];       b1 = theta[1];
    Wo = theta[2][:, 0]; bo = theta[3];
    # Compute number of parameters and initialize
    lw1 = len(W1.ravel());  lb1 = len(b1);
    lwo = len(Wo.ravel());  lbo = len(bo);
    d = W1.shape[1]    # Number of hidden layers
    nb_params = lw1 + lb1 + lwo + lbo;
    Hy = np.zeros((nb_params, nb_params))

    # Compute outputs of layers
    O1 = np.matmul(W1.T, X[0, :]) + b1
    yHat = np.dot(Wo, sig(O1)) + bo

    # Compute Hessian of output wrt params
    Hy[0:lw1, 0:lw1] = \
        np.kron(np.diag(ddsig(O1)*Wo),
                np.matmul(X.T, X))          # W1 / W1
    Hy[0:lw1, lw1:(lw1+lwo)] = \
        np.kron(np.diag(dsig(O1)), X).T     # W1 / Wo
    Hy[lw1:(lw1+lwo), 0:lw1] = Hy[0:lw1, lw1:(lw1+lwo)].T
    Hy[0:lw1, (lw1+lwo):(lw1+lwo+lb1)] = \
        np.kron(np.diag(ddsig(O1)*Wo), X).T # W1 / b1
    Hy[(lw1+lwo):(lw1+lwo+lb1), 0:lw1] = Hy[0:lw1, (lw1+lwo):(lw1+lwo+lb1)].T
    Hy[lw1:(lw1+lwo), (lw1+lwo):(lw1+lwo+lb1)] = \
        np.diag(dsig(O1))                   # Wo / b1
    Hy[(lw1+lwo):(lw1+lwo+lb1), lw1:(lw1+lwo)] = Hy[lw1:(lw1+lwo), (lw1+lwo):(lw1+lwo+lb1)].T
    Hy[(lw1+lwo):(lw1+lwo+lb1), (lw1+lwo):(lw1+lwo+lb1)] = \
        np.diag(ddsig(O1)*Wo)               # b1 / b1

    return (yHat - y)*Hy + np.matmul(grady, grady.T)  # SSE formula

def grady_i(theta, X, y):
    # Check if one sample is given
    if X.shape[0] > 1:
        print("Give one sample to the function with")
        print(" shape (1, p),")
        print(" where p is the number of features.")
        X = X[0:1, :]
        y = y[0].reshape(-1, 1)
    # Get parameters
    W1 = theta[0];       b1 = theta[1];
    Wo = theta[2][:, 0]; bo = theta[3];
    # Compute number of parameters and initialize
    lw1 = len(W1.ravel());  lb1 = len(b1);
    lwo = len(Wo.ravel());  lbo = len(bo);
    d = W1.shape[1]    # Number of hidden layers
    nb_params = lw1 + lb1 + lwo + lbo;
    grady = np.zeros((nb_params)).reshape(-1, 1)
    
    O1 = np.matmul(W1.T, X[0, :]) + b1
    # Compute gradient of output wrt params
    grady[0:lw1] = np.kron(dsig(O1)*Wo, X).T      # W1
    grady[lw1:(lw1+lwo)] = sig(O1).reshape(-1, 1) # Wo
    grady[(lw1+lwo):(lw1+lwo+lb1)] = (dsig(O1)*Wo).reshape(-1,1) # b1
    grady[(lw1+lwo+lb1):(lw1+lwo+lb1+lbo)] = 1    # bo

    return grady

#### Model settings

Définition du modèle pour un choix de $\eta$ et un choix de $m$ (nombre de neurones dans la couche cachée).
Calculera les dof avec la méthode de Ye (et enregistre) pour cette configuration.

L'ensemble des résultats, du document produit, nécessitent de répéter cette étape pour différentes tailles et différentes régularisation.

A souligner que les différents tests de l'estimateur proposés sont réalisés plus tard.

In [ ]:
set_seed(seed)

# Number of hidden layers of the perceptron model
k      = 1
# Number of neurons per hidden layer
m      = 2
# Reg number or position in l2_reg
eta    = 1e-12
# Learning settings
initial_learning_rate = 1.
final_learning_rate = 0.01
ep      = 600
t_size  = n
b_size  = 100

# Define model
model = Sequential()
for i in range(0,k):
    model.add(Dense(m, activation='tanh',kernel_initializer='he_normal',kernel_regularizer=l2(eta)))
model.add(Dense(1, activation='linear',kernel_regularizer=l2(eta)))

# Learning settings
learning_rate_decay_factor = (final_learning_rate / initial_learning_rate)**(1/ep)
steps_per_epoch = int(t_size/b_size)

lr_schedule =   ExponentialDecay(
                initial_learning_rate=initial_learning_rate,
                decay_steps=steps_per_epoch,
                decay_rate=learning_rate_decay_factor,
                staircase=True)
# Compile the model
sgd = SGD(learning_rate=lr_schedule)
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])
model.build(X.shape)
model.summary()
# Save weights
weights = model.get_weights()

Test sur un jeu de données

In [ ]:
# Init variables
mses_per = []
lambda_ = 1e-12
# Fit the model
for i in range(ep):
    history = model.fit(X, Y[:,0], epochs=1, batch_size=b_size, verbose=0)
    if i < 10 or (i+1) % 100 == 0:
        print("Epoch {} / {}  -  Loss: {:.4f} / mse: {:.4f}".format(
            i+1, ep, history.history['loss'][0], history.history['mse'][0]))
# Evaluate predictions
Y_pred = model.predict(X)
# Compute dofs
theta = [v.numpy() for v in model.trainable_variables]
nb_params = sum([len(v.ravel()) for v in theta])
# compute Hessian and Jacobians
H = np.zeros((nb_params, nb_params))
J_grad = np.zeros((nb_params, n))
Jy_m = np.zeros((n, nb_params))
# Loop on samples
for i in range(0, X.shape[0]):
    H += hessian_i(theta, X[i:i+1, :], Y[i, 0])
    Jy_m[i, :] = grady_i(theta, X[i:i+1, :], Y[i, 0]).T
    J_grad[:, i:(i+1)] = grady_i(theta, X[i:i+1, :], Y[i, 0])

# Normalize H (Frobenius norm)
froe_H = np.sqrt(np.trace(H.T.dot(H)))
H /= froe_H
J_grad /= -froe_H
# Compute the matrix to put within the trace as linear problem
H_inv_J_grad = np.linalg.lstsq(H + lambda_*np.eye(nb_params), J_grad, rcond=-1)[0] # rcond=None
Jy_y = -np.matmul(Jy_m, H_inv_J_grad)
# Compute dof - general formula with Gaussian iid data + Save MSE
df_ours.append(np.trace(Jy_y))
mses_per.append(MSE(Y_pred, Y[:, 0]))

print("\nDegrees of freedom : {} / Nb of parameters : {}".format(df_ours[0], nb_params))

Algorithm de Ye pour $\sigma$ connu et enregistré par sa vraie valeur.

In [ ]:
hatMu = np.zeros((n*n_mc, 1))
delta_mat = np.zeros((n*n_mc, n+1))
tau = 0.6*np.sqrt(sigma_2);              # std_deviation suggested by Ye, 1998 - \sigma is known
delta_mat[:, n] = 1

for i in range(n_mc):
    # Reinitialize model
    model.set_weights(weights)
    print('Monte-Carlo realization i: ',i)
    # Data perturbation
    delta_t = tau*np.random.randn(n).reshape(-1, 1)
    y_train_per = Y[:, 0].reshape(-1,1) + delta_t
    # Fit the model
    model.fit(X, y_train_per, epochs=ep,
              batch_size=b_size, verbose=0)
    # Evaluate predictions
    Y_pred = model.predict(X)
    # Save results for Ye
    hatMu[i::n_mc, :] = Y_pred            # See Ye, 1998
    delta_mat[i::n_mc, :n] = np.diag(delta_t[:,0])
    
    # Save MSE
    mses_per.append(MSE(Y_pred, Y[:, i]))


In [ ]:
delta_mat_sub = np.empty((n_mc, 2))
delta_mat_sub[:, 1] = 1;
# Ye solution
dof_Ye = 0

for k in range(0, n):
    # Fill the matrices
    delta_mat_sub[:, 0] = delta_mat[k*n_mc:(k+1)*n_mc, k]
    hatMu_sub = hatMu[k*n_mc:(k+1)*n_mc, :]
    # Compute dof
    sol_ = np.linalg.solve(delta_mat_sub.T.dot(delta_mat_sub) + lambda_*np.eye(2),
                           delta_mat_sub.T.dot(hatMu_sub))
    # Compute dof as the sum of the slopes
    dof_Ye += sol_[0]
    
# Print
print(dof_Ye[0])

#### Calcul de la convergence de l'algorithme Ye (étape par étape).

In [ ]:
print(int(n/2))
print(int(n/100))
print(int(n/50))

In [ ]:
dof_MC = np.zeros((n_mc))

for t in range(1, n_mc+1):
    # Extract indexes
    hatMu_sub = np.empty((t, 1))
    delta_mat_sub = np.empty((t, 2))
    delta_mat_sub[:, 1] = 1;
    
    # Loop
    for k in range(0, n):
        delta_mat_sub[:, 0] = delta_mat[k*n_mc:(k*n_mc+t), k]
        hatMu_sub = hatMu[k*n_mc:(k*n_mc+t), :]
        # Compute dof
        sol_ = np.linalg.solve(delta_mat_sub.T.dot(delta_mat_sub) + lambda_*np.eye(2),
                               delta_mat_sub.T.dot(hatMu_sub))
        dof_MC[t-1] += sol_[0]
    
    # print
    if t%20 == 0:
        print("Etape {} ok.".format(t))

Affichage de la convergence.

In [ ]:
plt.figure()
plt.plot(np.arange(1, n_mc+1), dof_MC)
plt.title("Convergence algo Ye")
plt.ylim(-5, 30)
plt.tight_layout()
plt.show()

Et affichage de l'erreur quadratique de chaque run.

In [ ]:
plt.figure()
plt.plot(list(range(1, n_mc+1)), mses_per[1:])
plt.title("MSE")
plt.tight_layout()
plt.show()

Sauvegarde dans .csv (dépend du nombre de neurones - voir format).

In [ ]:
dict_graph = {}

dict_graph["simus"]    = np.arange(1, n_mc+1)
dict_graph["conv_Ye"]  = dof_MC
dict_graph["dof"]      = np.array(df_ours[1:])   # Remove the first one (previous simulation)
dict_graph["MSE"]      = np.array(mses_per)

df = pd.DataFrame.from_dict(dict_graph, orient = "index")
df = df.T
df.to_csv("Dossier_resultats/MLP_results_nbHNeurons{}_reg{}.csv".format(m, int(-np.log10(eta))),
          na_rep="nan", index=False)

#### Calcul de notre estimateur pour différentes architectures

Boucle pour parcourir plusieurs dimensions et renvoyer les degrés de liberté.
$\eta$ est parcouru parmi "etas" (liste) et le nombre de neurones de la couche cachée varient entre 1 et "n_max".

En fin de boucle les résultats sont sauvegardés dans un fichier.

In [ ]:
etas   = [1e-2, 1e-3, 1e-6, 1e-12]
m_max  = 10             # Maximum number of hidden neurons
nb_params_mlp = []         # Save number of parameters

# Number of hidden layers of the perceptron model
k      = 1
# Learning settings
initial_learning_rate = 1.
final_learning_rate = 0.01
ep      = 600
t_size  = n
b_size  = 10
learning_rate_decay_factor = (final_learning_rate / initial_learning_rate)**(1/ep)
steps_per_epoch = int(t_size/b_size)

# Save dict
dict_graph = {}
# Loops
for eta in etas:       # Reg number or position in l2_reg
    mses = [];
    hat_df = [];
    gcvs =[];
    
    for m in range(1, m_max+1):   # For different neurons size
        set_seed(seed)  # reset seed

        # Define model
        model = Sequential()
        for i in range(0,k):
            model.add(Dense(m, activation='tanh',kernel_initializer='he_normal',kernel_regularizer=l2(eta)))
            model.add(Dense(1, activation='linear',kernel_regularizer=l2(eta)))

            lr_schedule =   ExponentialDecay(
                        initial_learning_rate=initial_learning_rate,
                        decay_steps=steps_per_epoch,
                        decay_rate=learning_rate_decay_factor,
                        staircase=True)
        # Compile the model
        sgd = SGD(learning_rate=lr_schedule)
        model.compile(optimizer='sgd', loss='mse', metrics=['mse'])
        model.build(X.shape)
        # model.summary()
        
        # Compute dofs
        lambda_ = 1e-10        # Regularization for inversion
        # Fit the model
        for i in range(ep):
            history = model.fit(X, Y[:,0], epochs=1, batch_size=b_size, verbose=0)
            if (i+1) == 500 or (i+1) == 600:
                print("Epoch {} / {}  -  Loss: {:.4f} / mse: {:.4f}".format(
                      i+1, ep, history.history['loss'][0], history.history['mse'][0]))
        # Evaluate predictions
        Y_pred = model.predict(X)
        # Compute dofs
        theta = [v.numpy() for v in model.trainable_variables]
        nb_params = sum([len(v.ravel()) for v in theta])
        # compute Hessian and Jacobians
        H = np.zeros((nb_params, nb_params))
        J_grad = np.zeros((nb_params, n))
        Jy_m = np.zeros((n, nb_params))
        # Loop on samples
        for i in range(0, X.shape[0]):
            H += hessian_i(theta, X[i:i+1, :], Y[i, 0])
            Jy_m[i:i+1, :] = grady_i(theta, X[i:i+1, :], Y[i, 0]).T
            J_grad[:, i:i+1] = grady_i(theta, X[i:i+1, :], Y[i, 0])
        # Normalize H (Frobenius norm)
        froe_H = np.sqrt(np.trace(H.T.dot(H)))
        H /= froe_H
        J_grad /= -froe_H
        # Compute the matrix to put within the trace as linear problem
        H_inv_J_grad = np.linalg.lstsq(H + lambda_*np.eye(nb_params), J_grad, rcond=None)[0]
        Jy_y = -np.matmul(Jy_m, H_inv_J_grad)
        # Compute dof - general formula with Gaussian iid data + Save MSE
        h_df = np.trace(Jy_y); mse_r = MSE(Y_pred, Y[:, 0]);
        hat_df.append(h_df)
        mses.append(mse_r)
        gcvs.append(n*mse_r/((n-h_df)*(n-h_df)))
        
        if eta == etas[0]:
            nb_params_mlp.append(nb_params)
            
        print("eta - {},  nb_neurons - {},  Done.".format(int(-np.log10(eta)), m))
            
    dict_graph["MSE_eps{}".format(int(-np.log10(eta)))] = np.array(mses)
    dict_graph["df_eps{}".format(int(-np.log10(eta)))]  = np.array(hat_df)
    dict_graph["GCV_eps{}".format(int(-np.log10(eta)))]  = np.array(gcvs)

dict_graph["nb_neurons"] = np.arange(1, m_max+1)
dict_graph["nb_params"]  = np.array(nb_params_mlp)

df = pd.DataFrame.from_dict(dict_graph, orient = "index")
df = df.T
df.to_csv("Dossier_resultats/MLP_results_hat_df.csv",
          na_rep="nan", index=False)

Fin de document.